<a href="https://colab.research.google.com/github/Jenoprabakar/computer-vision/blob/main/cv4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#IMAGE DENOISING
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error as mse
from scipy.ndimage import uniform_filter


img = cv2.imread("clean_image.jpg", cv2.IMREAD_GRAYSCALE)
img = img.astype(np.float32)

def gaussian_noise(image, mean=0, var=20):
    noise = np.random.normal(mean, np.sqrt(var), image.shape)
    return np.clip(image + noise, 0, 255)

def rayleigh_noise(image, scale=20):
    noise = np.random.rayleigh(scale, image.shape)
    return np.clip(image + noise, 0, 255)

def gamma_noise(image, shape=2, scale=10):
    noise = np.random.gamma(shape, scale, image.shape)
    return np.clip(image + noise, 0, 255)

def exponential_noise(image, scale=10):
    noise = np.random.exponential(scale, image.shape)
    return np.clip(image + noise, 0, 255)

def uniform_noise(image, low=-20, high=20):
    noise = np.random.uniform(low, high, image.shape)
    return np.clip(image + noise, 0, 255)

def salt_pepper_noise(image, prob=0.05):
    noisy = image.copy()
    rand = np.random.rand(*image.shape)
    noisy[rand < prob/2] = 0
    noisy[rand > 1 - prob/2] = 255
    return noisy


def mean_filter(image):
    return uniform_filter(image, size=3)

def median_filter(image):
    return cv2.medianBlur(image.astype(np.uint8), 3)

def gaussian_filter(image):
    return cv2.GaussianBlur(image, (3,3), 0)


noise_types = {
    "Gaussian": gaussian_noise(img),
    "Rayleigh": rayleigh_noise(img),
    "Gamma": gamma_noise(img),
    "Exponential": exponential_noise(img),
    "Uniform": uniform_noise(img),
    "Salt & Pepper": salt_pepper_noise(img)
}

filters = {
    "Mean": mean_filter,
    "Median": median_filter,
    "Gaussian": gaussian_filter
}


results = []

for noise_name, noisy_img in noise_types.items():
    best_psnr = 0
    best_filter = ""

    for filter_name, filter_func in filters.items():
        denoised = filter_func(noisy_img)
        p = psnr(img, denoised)
        m = mse(img, denoised)

        if p > best_psnr:
            best_psnr = p
            best_filter = filter_name

    results.append([noise_name, best_filter, round(best_psnr,2)])


plt.figure(figsize=(12,8))
i = 1
for noise_name, noisy_img in noise_types.items():
    plt.subplot(3,6,i)
    plt.title("Original")
    plt.imshow(img, cmap='gray')
    plt.axis('off')

    plt.subplot(3,6,i+1)
    plt.title(noise_name)
    plt.imshow(noisy_img, cmap='gray')
    plt.axis('off')

    plt.subplot(3,6,i+2)
    denoised = median_filter(noisy_img)
    plt.title("Denoised")
    plt.imshow(denoised, cmap='gray')
    plt.axis('off')
    i += 3

plt.tight_layout()
plt.show()



print("\nPerformance Table (Best Filter per Noise Type)")
print("-------------------------------------------------")
print("Noise Type       | Best Filter | PSNR (dB)")
print("-------------------------------------------------")
for r in results:
    print(f"{r[0]:15} | {r[1]:11} | {r[2]}")
